In [1]:
import requests
import json
import pandas as pd
 
headers = {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "no-cache",
    "content-type": "application/json",
    "expires": "0",
    "origin": "https://marketfiyati.org.tr",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-platform": '"Android"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "timeout": "20000",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36",
    "withcredentials": "true"
}
 
url = "https://api.marketfiyati.org.tr/api/v1/info/categories"
response = requests.get(url, headers=headers)
data = json.loads(response.text)
 
categories_list = []
 
for item in data['content']:
    subcategories = item['subcategories']
    for subcategory in subcategories:
        categories_list.append(subcategory)
 
 
search_results_df = pd.DataFrame()
 
for keyword in categories_list:
    page = 0 
    while True:
        post_data = {
            "keywords": keyword,
            "menuCategory": False,
            "pages": page,
            "size":200
        }
        response = requests.post("https://api.marketfiyati.org.tr/api/v2/searchByCategories", headers=headers, data=json.dumps(post_data))
        response_data = json.loads(response.text)
        if "content" in response_data and response_data["content"]:
            temp_df = pd.DataFrame(response_data["content"])
            temp_df["keyword"] = keyword
            search_results_df = pd.concat([search_results_df, temp_df], ignore_index=True)
            page += 1
        else:
            break
 
 
search_results_df
 
df_exploded = search_results_df.explode("productDepotInfoList", ignore_index=True)
product_depot_info_df = pd.json_normalize(df_exploded['productDepotInfoList'])
df_exploded = pd.concat([df_exploded, product_depot_info_df], axis=1)
df_exploded.drop(columns=["productDepotInfoList"], inplace=True)
df_exploded
df_exploded.drop(["categories","imageUrl","percentage","longitude","latitude"],axis=1).to_excel("comparison.xlsx")
 

In [2]:
df_exploded

,id,title,brand,imageUrl,categories,keyword,depotId,depotName,price,marketAdi,percentage,longitude,latitude,indexTime
0,1d8a522c-7432-47bc-b0c0-642d80b1b4aa,Cennet Hurma Kuru 500 gr,Sivas Birlik,https://api.marketfiyati.org.tr/api/v1/resourc...,"[Meyve Ve Sebzeler, Normal Meyveler, Normal Me]",Meyve,hakmar-5690,Acıbadem Kadıköy,189.95,hakmar,0.000000,29.036860,40.997680,11.02.2025 07:18
1,030602da-0213-492a-842d-b2cfcb8cd6b4,Mango Adet,Markasız,https://cdn.cimri.io/market/500x500/-_87469.jpg,"[Meyveler, Meyve Ve Sebzeler, Egzoti̇k Meyvele...",Meyve,hakmar-5200,Valide Üsküdar,44.95,hakmar,0.000000,29.031761,41.018707,11.02.2025 07:18
2,09465ef3-ae27-47ac-b8f8-b1a5dfaf2abd,Portakal,Markasız,https://cdn.cimri.io/market/500x500/-_218863.jpg,"[Meyve Ve Sebzeler, Meyveler, Narenciye, Meyve...",Meyve,tarim_kredi-5022,Istanbul Kadıköy Market,29.90,tarim_kredi,0.000000,29.043560,40.988700,11.02.2025 07:18
3,09465ef3-ae27-47ac-b8f8-b1a5dfaf2abd,Portakal,Markasız,https://cdn.cimri.io/market/500x500/-_218863.jpg,"[Meyve Ve Sebzeler, Meyveler, Narenciye, Meyve...",Meyve,sok-1017,Selimiye,34.75,sok,16.220737,29.017970,41.010906,11.02.2025 07:18
4,09465ef3-ae27-47ac-b8f8-b1a5dfaf2abd,Portakal,Markasız,https://cdn.cimri.io/market/500x500/-_218863.jpg,"[Meyve Ve Sebzeler, Meyveler, Narenciye, Meyve...",Meyve,a101-4217,Rıhtım Kadıköy Istanbul,34.75,a101,16.220737,29.026400,40.994100,11.02.2025 07:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14658,78ca62a5-688e-4de2-acc5-e526d314282d,Yumak Kare Makyaj Temizleme Pamuğu 70 li,Yumak,https://api.marketfiyati.org.tr/api/v1/resourc...,"[Makyaj Pamuğu, Pamuk, Kişisel Bakım, Kağıt]",Diğer Temizlik ve Kişisel Bakım Ürünleri,sok-1017,Selimiye,37.00,sok,0.000000,29.017970,41.010906,11.02.2025 07:18
14659,36493921-0943-4485-aa0a-221f9c5317be,Ayak Mas.soyulabilen 30mldermokil,Dermokil,https://api.marketfiyati.org.tr/api/v1/resourc...,"[Güzellik, Kozmetik Ve Kişisel Bakım, Güzellik...",Diğer Temizlik ve Kişisel Bakım Ürünleri,hakmar-5200,Valide Üsküdar,34.50,hakmar,0.000000,29.031761,41.018707,11.02.2025 07:18
14660,ec1f5aed-45c3-48ff-af5d-cecc09c44c3c,Getwell Pırıl Yara Bandı,Getwell,https://reimg-carrefour.mncdn.com/mnresize/600...,"[Gıda, Parfümerı Ve Hıjyen, Ilk Yardım Ürünler...",Diğer Temizlik ve Kişisel Bakım Ürünleri,carrefour-5254,Istanbul Üsküdar Salacak Mını,16.20,carrefour,0.000000,29.011852,41.018646,11.02.2025 07:18
14661,69b04cf6-fe43-4dc7-80b3-00116910606a,Yumak Disk Pamuk 70 Adet,Yumak,https://api.marketfiyati.org.tr/api/v1/resourc...,"[Pamuk, Kişisel Bakım, Disk Pamuk, Kağıt]",Diğer Temizlik ve Kişisel Bakım Ürünleri,sok-1017,Selimiye,20.00,sok,0.000000,29.017970,41.010906,11.02.2025 07:18
